In [5]:
pip install geopandas

     ---------------------------------------- 1.1/1.1 MB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 6.1/6.1 MB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 22.9/22.9 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install dbfread

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF

In [ ]:
#PASO 1 - Delimitación del área de estudio

area_estudio = gpd.read_file("ruta/al/shapefile_o_GeoJSON")

In [22]:
#PASO DOS - Preparación de información censal

# Función para cargar datos de archivo DBF y convertirlos a DataFrame
def load_dbf_to_dataframe(file_path):
    table = DBF(file_path)
    return pd.DataFrame(iter(table))

# Cargar archivos en un DataFrame
MZNA_2020_df = pd.read_csv(r"C:\Users\Alejandro\Downloads\DATA\2020\MZNA\RESAGEBURB_11CSV20.csv")
ITER_2020_df = pd.read_csv(r"C:\Users\Alejandro\Downloads\DATA\2020\ITER\ITER_NALCSV20.csv")

MZNA_2010_df = load_dbf_to_dataframe(r"C:\Users\Alejandro\Downloads\DATA\2010\MZNA\RESAGEBURB_11DBF10.dbf")
ITER_2010_df = load_dbf_to_dataframe(r"C:\Users\Alejandro\Downloads\DATA\2010\ITER\ITER_NALDBF10.dbf")

In [ ]:
# 2.2 Homologación de ID urbana
# Creamos el campo CVEGEO concatenando claves.
def homologate_urban_id(row):
    return f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}{row['AGEB']}{row['MZA']}"

# 3.2 Homologación de ID rural
def homologate_rural_id(row):
    return f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}0000000"

MZNA_2020_df['CVEGEO'] = MZNA_2020_df.apply(homologate_urban_id, axis=1)
MZNA_2010_df['CVEGEO'] = MZNA_2010_df.apply(homologate_urban_id, axis=1)

ITER_2020_df['CVEGEO'] = ITER_2020_df.apply(homologate_rural_id, axis=1)
ITER_2010_df['CVEGEO'] = ITER_2010_df.apply(homologate_rural_id, axis=1)

In [31]:
#reemplazar *
def replace(dataframe):
    return dataframe.replace('*', '0')

replace(MZNA_2020_df)
replace(MZNA_2010_df)
replace(ITER_2020_df)
replace(ITER_2010_df)

In [ ]:
# Función para filtrar información por municipios correspondientes
def filtrar municipios(dataframe, municipios):
    return dataframe[dataframe['CVEGEO'].isin(municipios)]

municipios = ['002', '039', '044', '051' , '070', '097', '098', '101', '120', '124']

# aplicar filtrado por municipios
MZNA_2020_df_filter = filter_by_municipios(MZNA_2020_df, municipios)
MZNA_2010_df_filter = filter_by_municipios(MZNA_2010_df, municipios)
ITER_2020_df_filter = filter_by_municipios(ITER_2020_df, municipios)
ITER_2010_df_filter = filter_by_municipios(ITER_2010_df, municipios)

In [29]:
# 2.3 Guardado de información en CSV
def save_to_csv(dataframe, filename):
    dataframe.to_csv(filename, index=False)
    
save_to_csv(MZNA_2020_df, r"C:\Users\Alejandro\Downloads\DATA\Tratados\CONCATENADOS\MZNA_2020_CONCAT.csv")
save_to_csv(MZNA_2010_df, r"C:\Users\Alejandro\Downloads\DATA\Tratados\CONCATENADOS\MZNA_2010_CONCAT.csv")
save_to_csv(ITER_2020_df, r"C:\Users\Alejandro\Downloads\DATA\Tratados\CONCATENADOS\ITER_2020_CONCAT.csv")
save_to_csv(ITER_2010_df, r"C:\Users\Alejandro\Downloads\DATA\Tratados\CONCATENADOS\ITER_2010_CONCAT.csv")

In [ ]:
#PASO TRES - Unión de información estadística y cartografía

#Carga de información estadística y cartográfica
SHP_MZN = r"C:\Users\Alejandro\Downloads\DATA\SHP\11m.shp
SHP_LOC = r"C:\Users\Alejandro\Downloads\DATA\SHP\11l.shp"

# Utilizamos geopandas para cargar datos geoespaciales.
SHP_MZN_gdf = gpd.read_file(SHP_MZN)
SHP_LOC_gdf = gpd.read_file(SHP_LOC)

In [ ]:
# 4.2 Filtrado de información por municipio
municipios = ['002', '039', '044', '051' , '070', '097', '098', '101', '120', '124']

SHP_MZN_gdf_filter = SHP_MZN_gdf[SHP_MZN_gdf['CVEGEO'].isin(municipios)]
SHP_LOC_gdf_filter = SHP_LOC_gdf[SHP_LOC_gdf['CVEGEO'].isin(municipios)]



In [ ]:
# 4.3 Guardado de nueva capa principal
MAPA_TIPO_gpkg = "Mapa_Tipologias.gpkg"
SHP_MZN_gdf_filter.to_file(MAPA_TIPO_gpkg, layer='Mapa_Tipologias', driver='GPKG')

In [ ]:
# 4.4 Selección de ITER rural y unión de información
SHP_LOC_gdf_filter_RURAL = gdf_iter[gdf_iter['TIPO'] == 'Rural']

# Guardar los elementos filtrados en el GeoPackage base
SHP_LOC_gdf_filter_RURAL.to_file(MAPA_TIPO_gpkg, layer='Mapa_Tipologias', driver='GPKG', append=True)

In [ ]:
# Leer el GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_gpkg)

# Unir campos de los DataFrames al GeoPackage base por CVEGEO
MAPA_TIPO_gdf = MAPA_TIPO_gdf.merge(MZNA_2020_df[['CVEGEO', 'POBTOT', 'VIVPAR', 'VIVPAR_DES']], on='CVEGEO', how='left')
MAPA_TIPO_gdf = MAPA_TIPO_gdf.merge(ITER_2020_df[['CVEGEO', 'POBTOT', 'VIVPAR']], on='CVEGEO', how='left')
MAPA_TIPO_gdf = MAPA_TIPO_gdf.merge(MZNA_2010_df[['CVEGEO', 'POBTOT', 'VIVPAR', 'VIVPAR_DES']], on='CVEGEO', how='left')
MAPA_TIPO_gdf = MAPA_TIPO_gdf.merge(ITER_2010_df[['CVEGEO', 'POBTOT', 'VIVPAR']], on='CVEGEO', how='left')

# Crear campos POPTOT2010, POPTOT2020, VIVPAR2010 y VIVPAR2020 y llenado respectivamente
MAPA_TIPO_gdf['POPTOT2010'] = MAPA_TIPO_gdf['POBTOT_x'].fillna(0) + MAPA_TIPO_df['POBTOT_y'].fillna(0)
MAPA_TIPO_gdf['POPTOT2020'] = MAPA_TIPO_gdf['POBTOT'].fillna(0)
MAPA_TIPO_gdf['VIVPAR2010'] = MAPA_TIPO_gdf['VIVPAR_x'].fillna(0) + gdf_base['VIVPAR_DES'].fillna(0) + gdf_base['VIVPAR_y'].fillna(0)
MAPA_TIPO_gdf['VIVPAR2020'] = MAPA_TIPO_gdf['VIVPAR'].fillna(0)

# Actualizar valores sin información de "null" a 0
MAPA_TIPO_gdf = gdf_base.fillna(0)

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf.to_file(MAPA_TIPO_gpkg, layer='Mapa_Tipologias', driver="GPKG")


In [ ]:
#PASO CUATRO - Creación de los campos necesarios para el mapa final
#PASO CINCO - Llenado de informacióN

# Paso 5: Creación de campos y llenado de información
MAPA_TIPO_gdf['DIFPOP'] = MAPA_TIPO_gdf['POPTOT2020'] - MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['DIFVIV'] = MAPA_TIPO_gdf['VIVTOT2020'] - MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['RELPOP'] = MAPA_TIPO_gdf['DIFPOP'] / MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['RELVIV'] = MAPA_TIPO_gdf['DIFVIV'] / MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['CVEPOP'] = np.where(MAPA_TIPO_gdf['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_gdf['CVEVIV'] = np.select(
    [MAPA_TIPO_gdf['RELVIV'] < -0.075, (MAPA_TIPO_gdf['RELVIV'] >= -0.075) & (MAPA_TIPO_gdf['RELVIV'] <= 0.075), MAPA_TIPO_gdf['RELVIV'] > 0.075],
    [1, 2, 3]
)
MAPA_TIPO_gdf['CVE'] = MAPA_TIPO_gdf['CVEPOP'] + MAPA_TIPO_gdf['CVEVIV'].astype(str)
MAPA_TIPO_gdf['TIPOLOGIA'] = np.where((MAPA_TIPO_gdf['POPTOT2020'] == 0) & (MAPA_TIPO_gdf['POPTOT2010'] == 0) & (MAPA_TIPO_gdf['VIVTOT2010'] == 0) & (MAPA_TIPO_gdf['TIPOLOGIA'] == 'B3'), 'B3', MAPA_TIPO_gdf['TIPOLOGIA'])
MAPA_TIPO_gdf['area_ha'] = gdf_merged.area / 10000
MAPA_TIPO_gdf['FILTRADO'] = gdf_merged['VIVTOT2020'] / gdf_merged['area_ha']**2


In [ ]:
#PASO SEIS - Llenado de informacióN

# Paso 6: Filtrado de información
MAPA_TIPO_gdf_FILTRO = MAPA_TIPO_gdf[MAPA_TIPO_gdf['FILTRADO'] >= 0.125]

In [ ]:
# Guardar el resultado final
MAPA_TIPO_FILTRO_gpkg = "Mapa_Tipologias_Filtrado.gpkg"
MAPA_TIPO_gdf_FILTRO.to_file(MAPA_TIPO_FILTRO_gpkg, layer='Mapa_Tipologias_Filtrado', driver='GPKG')

In [ ]:
# Paso 7: Representación gráfica
import matplotlib.pyplot as plt

def plot_map_with_legend(gdf, colormap, legend_labels, legend_title, save_path):
    fig, ax = plt.subplots(1, 1)
    
    # Mapear cada categoría a su color correspondiente
    for category, color in colormap.items():
        gdf[gdf['TIPOLOGIA'] == category].plot(ax=ax, color=color, label=category, edgecolor='none')
    
    # Crear leyenda
    handles = [plt.Rectangle((0,0),1,1, color=color) for color in colormap.values()]
    ax.legend(handles, legend_labels, title=legend_title)
    
    # Eliminar línea de contorno
    ax.axis('off')
    
    # Guardar el mapa como imagen
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()

# Definir el diccionario de colores
colormap = {
    'DH': '#882A1C',
    'A1': '#E33F22',
    'A2': '#E5B636',
    'A3': '#FAF36A',
    'B1': '#CAEA60',
    'B2': '#60D512',
    'B3': '#39781C',
    'B3\'': '#254D12'
}

# Definir etiquetas para la leyenda
legend_labels = list(colormap.keys())

# Título de la leyenda
legend_title = 'Tipología'

# Ruta donde se guardará el mapa
map_image_path = "mapa_tipologias.png"

# Llamar a la función para generar y guardar el mapa
plot_map_with_legend(gdf_filtered, colormap, legend_labels, legend_title, map_image_path)
